<a href="https://colab.research.google.com/github/giuseppefutia/ml/blob/master/rgcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

# Set up the system

In [1]:
import torch
import numpy as np

dtype = torch.float
device = torch.device("cpu")
if torch.cuda.is_available(): # Try to use GPU if available
  device = torch.device('cuda')

print('Your device is ' + str(device))

Your device is cuda


# Import external files


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving __init.py__ to __init.py__
Saving data_generator.py to data_generator.py
Saving data_importer.py to data_importer.py
Saving data_reader.py to data_reader.py
Saving entities.dict to entities.dict
Saving relations.dict to relations.dict
Saving settings_importer.py to settings_importer.py
Saving settings.json to settings.json
Saving test.txt to test.txt
Saving train.txt to train.txt
Saving valid.txt to valid.txt
User uploaded file "__init.py__" with length 2 bytes
User uploaded file "data_generator.py" with length 1953 bytes
User uploaded file "data_importer.py" with length 947 bytes
User uploaded file "data_reader.py" with length 923 bytes
User uploaded file "entities.dict" with length 749 bytes
User uploaded file "relations.dict" with length 1543 bytes
User uploaded file "settings_importer.py" with length 1048 bytes
User uploaded file "settings.json" with length 1291 bytes
User uploaded file "test.txt" with length 775 bytes
User uploaded file "train.txt" with length 805 bytes
Use

# Prepare data

The preparation of data consist in the following steps:

1.   TODO
2.   TODO




In [8]:
from data_reader import *

relations_path = 'relations.dict'
entities_path = 'entities.dict'
train_path = 'train.txt'
test_path = 'test.txt'
valid_path = 'valid.txt'

# Create a list of triples using their id
train = read_triplets_as_list(train_path, entities_path, relations_path)
test = read_triplets_as_list(test_path, entities_path, relations_path)
valid = read_triplets_as_list(valid_path, entities_path, relations_path)

# Create a numpy matrix using the previous list
train = np.array(train)
test = np.array(test)
valid = np.array(valid)

# Create a pytorch tensor from numpy matrix
train = torch.from_numpy(train)
test  = torch.from_numpy(test)
valid = torch.from_numpy(valid)

# Read entities and relations as dictionary
entities = read_dictionary(entities_path)
relations = read_dictionary(relations_path)

print('Data for training: ')
print(train)

print()

print('Data for testing: ')
print(test)

print()

print('Data for validation: ')
print(valid)

print()






Data for training: 
tensor([[19,  8, 20],
        [21,  9, 22],
        [23, 10, 24],
        [25, 11, 26],
        [27, 12, 28],
        [29, 13, 30],
        [31, 14, 32],
        [33, 15, 34],
        [35, 16, 36],
        [37,  7, 38]])

Data for testing: 
tensor([[ 0,  0,  1],
        [ 2,  1,  3],
        [ 4,  2,  5],
        [ 6,  3,  7],
        [ 8,  0,  9],
        [10,  4, 11],
        [12,  5, 13],
        [14,  6, 15],
        [14,  6, 16],
        [17,  7, 18]])

Data for validation: 
tensor([[39, 17, 40],
        [41, 18, 42],
        [43, 11, 44],
        [45, 19, 46],
        [47, 20, 48],
        [49, 21, 50],
        [51,  4, 52],
        [53, 22, 54],
        [55, 23, 56],
        [57, 24, 58]])



# Import settings

In [4]:
import json
from settings_importer import *

with open('settings.json') as f:
    settings= json.load(f)

encoder_settings, \
decoder_settings, \
optimizer_settings, \
evaluation_settings, \
general_settings = import_settings(settings, entities, relations, train)

print('Neural Network settings: ')
print()
print('Encoder settings: ')
print(encoder_settings)
print()
print('Decoder settings: ')
print(decoder_settings)
print()
print('Optimizer settings: ')
print(optimizer_settings)
print()
print('Evaluation settings: ')
print(evaluation_settings)
print()
print('General settings: ')
print(general_settings)

Neural Network settings: 

Encoder settings: 
{'Name': 'gcn_basis', 'DropoutKeepProbability': 0.8, 'InternalEncoderDimension': 500, 'NumberOfBasisFunctions': 5, 'NumberOfLayers': 2, 'UseInputTransform': 'Yes', 'UseOutputTransform': 'No', 'AddDiagonal': 'No', 'DiagonalCoefficients': 'No', 'SkipConnections': 'No', 'StoreEdgeData': 'No', 'RandomInput': 'No', 'PartiallyRandomInput': 'No', 'Concatenation': 'No', 'NegativeSampleRate': 10, 'GraphSplitSize': 0.5, 'ExperimentName': 'models/GcnBlock', 'GraphBatchSize': 3, 'EntityCount': 59, 'RelationCount': 25, 'EdgeCount': 10, 'CodeDimension': 500}

Decoder settings: 
{'Name': 'bilinear-diag', 'RegularizationParameter': 0.01, 'NegativeSampleRate': 10, 'GraphSplitSize': 0.5, 'ExperimentName': 'models/GcnBlock', 'GraphBatchSize': 3, 'EntityCount': 59, 'RelationCount': 25, 'EdgeCount': 10, 'CodeDimension': 500}

Optimizer settings: 
{'MaxGradientNorm': 1, 'ReportTrainLossEvery': 100, 'EarlyStopping': {'CheckEvery': 100, 'BurninPhaseDuration': 6000

# Build the Model

The Graph Convolutional Network in our case is an encoder.

## Help Functions for weight initialization

The following functions are useful for the correct initialization of weights.

### Glorot initialization

The Glorot initialization, for instance, makes sure the weights are ‘just right’, keeping the signal in a reasonable range of values through many layers. Glorot initialization automatically determines the scale of initialization based on the number of input and output neurons.

A useful explanation is available at: http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization.




In [0]:
def glorot_variance(shape):
    return 3 / np.sqrt(shape[0] + shape[1])


def make_weight(mean, variance, shape, init="normal"):
    if init == "normal":
        initializer = np.random.normal(mean, variance, size=shape).astype(np.float32)
    elif init == "uniform":
        initializer = np.random.uniform(mean, variance, size=shape).astype(np.float32)

    return torch.from_numpy(initializer)


def make_bias(shape, init=0):
    if init == 0:
        return torch.from_numpy(np.zeros(shape).astype(np.float32))
    elif init == 1:
        return torch.from_numpy(np.ones(shape).astype(np.float32))

The RGCN model takes as input the encoder settings that contain all parameters to define the weight matrices of the encoder component.

In [0]:
class RGCN(torch.nn.Module):
  
  def __init__(self, encoder):
    
    super(RGCN, self).__init__()
    
    # Affine Layer 
    self.input_shape = [int(encoder_settings['EntityCount']),
                        int(encoder_settings['InternalEncoderDimension'])]
    
    self.affine = torch.nn.Linear(input_shape[0], input_shape[1])
    
    # Basic GCN Layers
    self.internal_shape = [int(encoder_settings['InternalEncoderDimension']),
                           int(encoder_settings['InternalEncoderDimension'])]
    
    # Features dimension of the vertex representation is equal to the number of entities
    vertex_feature_dimension = input_shape[0]
    n_coefficients = int(encoder_settings['NumberOfBasisFunctions'])
    relation_count = int(encoder_settings['RelationCount'])
          
    type_matrix_shape = (relation_count, n_coefficients)
    vertex_matrix_shape = (vertex_feature_dimension, n_coefficients,  self.internal_shape[1])
    self_matrix_shape = (vertex_feature_dimension,  self.internal_shape[1])

    glorot_var_combined = glorot_variance([vertex_matrix_shape[0], vertex_matrix_shape[2]])
        
        
        
        self.W_forward = make_tf_variable(0, glorot_var_combined, vertex_matrix_shape, name="GCN_Basis_W_forward")
        self.W_backward = make_tf_variable(0, glorot_var_combined, vertex_matrix_shape, name="GCN_Basis_W_backward")
        self.W_self = make_tf_variable(0, glorot_var_combined, self_matrix_shape, name="GCN_Basis_W_self")

        type_init_var = 1
        self.C_forward = make_tf_variable(0, type_init_var, type_matrix_shape, name="GCN_Basis_C_forward")
        self.C_backward = make_tf_variable(0, type_init_var, type_matrix_shape, name="GCN_Basis_C_backward")

        self.b = make_tf_bias(self.shape[1], name="GCN_Basis_bias")
    
    # Relation Embedding Layer
        
  def forward(X):
    
    # Set up the Affine Layer
    variance = glorot_variance(self.input_shape)
    affine.weight.data.fill_(make_weight(0, variance, input_shape))
    affine.bias.data.fill(make_bias(input_shape[1]))
      